__Problem 43__
----
Given the table below, called 'orders', write code to show the average revenue by month by channel:

In [53]:
import pandas as pd
raw_data = {'order_id': [1,2,3,4,5,6],
            'channel': ['Online', 'Online', 'In_store', 'In_store','Online','Online'],
            'date': ['2018-09-01', '2018-09-03', '2018-10-11', "2018-08-21","2018-08-13",'2018-10-29'],
            'month': ['09', '09', '10', '08','08','10'],
            'revenue': [100,125,200,80,200,100]}
df = pd.DataFrame(raw_data, columns = ['order_id', 'channel', 'date','month', 'revenue'])
df

,order_id,channel,date,month,revenue
0,1,Online,2018-09-01,09,100
1,2,Online,2018-09-03,09,125
2,3,In_store,2018-10-11,10,200
3,4,In_store,2018-08-21,08,80
4,5,Online,2018-08-13,08,200
5,6,Online,2018-10-29,10,100


Your result should return the following in a structured table:

<pre>Month | Channel | Avg. Revenue</pre>

_____
Solution

In [2]:
df_new = df.groupby(['month', 'channel'])['revenue'].mean().reset_index()
df_new.rename(columns={'revenue':'avg_revenue'})

,month,channel,avg_revenue
0,08,In_store,80.0
1,08,Online,200.0
2,09,Online,112.5
3,10,In_store,200.0
4,10,Online,100.0


Problem 44 
-----
Suppose you work for a retail company that has a database containing two tables, one called 'orders', and one called 'revenue', as shown below:

In [20]:
import sqlite3
conn = sqlite3.connect(':memory:')
crsr = conn.cursor()

def create_table_sql() -> str:
    return """
    CREATE TABLE Orders (
    order_id INTEGER,
    channel VARCHAR(1024),
    date VARCHAR(1024),
    month VARCHAR(1024)
    );
    
    CREATE TABLE Revenue (
        order_id INTEGER,
        revenue INTEGER
    );
    """

crsr.executescript(create_table_sql())
conn.commit()

In [21]:
def insert_data_sql() -> str:
    return """
    INSERT or IGNORE INTO Orders
    (order_id, channel, date, month)
    VALUES
    (1, "Online", "2018-09-01", "September"),
    (2, "Online", "2018-09-03", "September"),
    (3, "In_store", "2018-10-11", "October"),
    (4, "In_store", "2018-08-21", "August"),
    (5, "Online", "2018-08-13", "August"),
    (6, "Online", "2018-10-29", "October");
    
    INSERT or IGNORE INTO Revenue
    (order_id, revenue)
    VALUES
    (1, 100),
    (2, 125),
    (3, 200),
    (4, 80),
    (5, 200),
    (6, 100);
    """

crsr.executescript(insert_data_sql())
conn.commit()

In [50]:
def grab_order_table() -> str:
    return """
    SELECT *
    FROM Orders;
    """

crsr.execute(grab_order_table())
orders = crsr.fetchall()

pd.DataFrame(orders, columns=['order_id', 
                              'channel', 
                              'date', 
                              'month'])

,order_id,channel,date,month
0,1,Online,2018-09-01,September
1,2,Online,2018-09-03,September
2,3,In_store,2018-10-11,October
3,4,In_store,2018-08-21,August
4,5,Online,2018-08-13,August
5,6,Online,2018-10-29,October


In [51]:
def grab_revenue_table() -> str:
    return """
    SELECT *
    FROM Revenue;
    """

crsr.execute(grab_revenue_table())
revenue = crsr.fetchall()

pd.DataFrame(revenue, columns=['order_id', 'revenue'])

,order_id,revenue
0,1,100
1,2,125
2,3,200
3,4,80
4,5,200
5,6,100


In [52]:
def get_sql() -> str:
    return """
    SELECT o.channel,
           SUM(r.revenue) AS total_revenue
    FROM Orders o
    JOIN Revenue r
    ON o.order_id = r.order_id
    WHERE o.month IN ("September", "October")
    GROUP BY o.channel
    """
crsr.execute(get_sql2())
result = crsr.fetchall()
pd.DataFrame(result, columns=['channel', 'total_revenue'])

,channel,total_revenue
0,In_store,200
1,Online,325


Problem 42
-----

You are given an array containing reviews for a popular iOS app below:

reviews = [‘app is good, but forced updates are too frequent‘, ‘love this app, use it daily to log calories', 'free version of this app has way too many ads’, ‘app doesn't load, 0/10’]

Your task is to determine sentiment from the review above. To do this you first decide to write code to find the count of individual words across all the reviews -- write this code using Python.

In [1]:
def word_dist(reviews):
    word_count = {}
    
    for review in reviews:
        words = review.split(' ')
        for word in words:
            if word not in word_count:
                word_count[word] = 1
            else:
                word_count[word] += 1
    return word_count

In [2]:
reviews = ['app is good, but forced updates are too frequent', 
           'love this app, use it daily to log calories', 
           'free version of this app has way too many ads', 
           "app doesn't load, 0/10"]

word_dist(reviews)

{'app': 3,
 'is': 1,
 'good,': 1,
 'but': 1,
 'forced': 1,
 'updates': 1,
 'are': 1,
 'too': 2,
 'frequent': 1,
 'love': 1,
 'this': 2,
 'app,': 1,
 'use': 1,
 'it': 1,
 'daily': 1,
 'to': 1,
 'log': 1,
 'calories': 1,
 'free': 1,
 'version': 1,
 'of': 1,
 'has': 1,
 'way': 1,
 'many': 1,
 'ads': 1,
 "doesn't": 1,
 'load,': 1,
 '0/10': 1}

Problem 40
----
Given the table below, called 'orders', write a SQL query to show the monthly revenue growth. To calculate the monthly revenue growth, you can apply the following logic:

revenue growth = (current month's revenue-prior month's revenue)/prior month's revenue.



In [48]:
import sqlite3
conn = sqlite3.connect(':memory:')
crsr = conn.cursor()

def create_table_sql() -> str:
    return """
    CREATE TABLE Orders (
    order_id INTEGER,
    channel VARCHAR(1024),
    date VARCHAR(1024),
    month INTEGER,
    revenue FLOAT
    );
    """

crsr.executescript(create_table_sql())
conn.commit()

In [49]:
def insert_data_sql() -> str:
    return """
    INSERT or IGNORE INTO Orders
    (order_id, channel, date, month, revenue)
    VALUES
    (1, "Online", "2018-09-01", 9, 100),
    (2, "Online", "2018-09-03", 9, 125),
    (3, "In_store", "2018-10-11", 10, 200),
    (4, "In_store", "2018-08-21", 8, 80),
    (5, "Online", "2018-08-13", 8, 200);
    """

crsr.executescript(insert_data_sql())
conn.commit()

In [55]:
def get_sql() -> str:
    return """
    SELECT s.month,
           (s.current_rev-s.prior_rev)/s.prior_rev
    FROM (
        SELECT month,
               SUM(revenue) AS current_rev,
               LAG(SUM(revenue),1) OVER (ORDER BY month) AS prior_rev
        FROM Orders
        GROUP BY month
        ORDER BY month
    ) s
    """
crsr.execute(get_sql())
result = crsr.fetchall()
pd.DataFrame(result, columns=['month', 'grow_rate'])

,month,grow_rate
0,8,NaN
1,9,-0.196429
2,10,-0.111111


Problem 39
----
You are given a dataframe containing student information, named df (shown below).
Suppose you want to normalize each student's grade based on their age.

In [56]:
import pandas as pd

raw_data = {'name': ['Willard Morris', 'Al Jennings', 'Omar Mullins', 'Spencer McDaniel'],
                'age': [20, 19, 22, 21],
                'favorite_color': ['blue', 'blue', 'yellow', "green"],
                'grade': [88, 92, 95, 70]}

df = pd.DataFrame(raw_data)
df.head()

,name,age,favorite_color,grade
0,Willard Morris,20,blue,88
1,Al Jennings,19,blue,92
2,Omar Mullins,22,yellow,95
3,Spencer McDaniel,21,green,70


Write a function using Python Pandas that will add a new column to your dataframe containing a new grade normalized against the mean age of the students.


In [59]:
df['grade_norm'] = df['grade']*(df['age']/df['age'].mean())

In [60]:
df

,name,age,favorite_color,grade,grade_norm
0,Willard Morris,20,blue,88,85.853659
1,Al Jennings,19,blue,92,85.268293
2,Omar Mullins,22,yellow,95,101.951220
3,Spencer McDaniel,21,green,70,71.707317
